# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Kafka Producer: GPS Location Producer** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

### Invoke the transaction data generator

In [4]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 5.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from gatubelxs.page_views import generate_page_view
from gatubelxs.click_events import generate_click_event
from gatubelxs.user_interaction import generate_user_interaction
from kafka import KafkaProducer
from random import randint
import json
import time

In [2]:
KAFKA_SERVER = '3e6802a385a9:9093'
TOPIC_PAGE_VIEWS = "page_views"
TOPIC_CLICK_EVENTS = "click_events"
TOPIC_USER_INTERACTIONS = "user_interactions"

NUMBER_SESSION = 100 #Number of web sessions to simulate
MAX_EVENTS_PER_SESSION = 20

In [3]:
#Producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_SERVER,
    value_serializer=lambda m: json.dumps(m).encode('utf-8')
)

In [4]:
import uuid
from random import randint

def simulate_session():
    user_id = str(uuid.uuid4())
    session_id = str(uuid.uuid4())

    n_page_views = randint(1, MAX_EVENTS_PER_SESSION)
    n_clicks = randint(0, MAX_EVENTS_PER_SESSION)
    n_interactions = randint(0, MAX_EVENTS_PER_SESSION)

    for _ in range(n_page_views):
        evt = generate_page_view(user_id, session_id)
        producer.send(TOPIC_PAGE_VIEWS, evt)
        print(f"[{TOPIC_PAGE_VIEWS}] ->", evt)
    
    for _ in range(n_clicks):
        evt = generate_click_event(user_id, session_id)
        producer.send(TOPIC_CLICK_EVENTS, evt)
        print(f"[{TOPIC_CLICK_EVENTS}] ->", evt)
    
    for i in range(n_interactions):
        evt = generate_user_interaction(user_id, session_id)
        producer.send(TOPIC_USER_INTERACTIONS, evt)
        print(f"[{TOPIC_USER_INTERACTIONS}] ->", evt)

In [5]:
try:
    print(f"Starting to produce sessions...")
    for i in range(NUMBER_SESSION):
        simulate_session()
        time.sleep(2)
except KeyboardInterrupt:
    print("Interrumpted by user; stopping.")
finally:
    producer.flush()
    producer.close()

Starting to produce sessions...
[page_views] -> {'user_id': '4409a7c3-2ef1-4a2f-813d-6e65212cafc6', 'session_id': 'bd9a1ad0-910f-4150-93f7-c87b10f59f9b', 'page_url': '/product/1112', 'referrer_url': 'tag/search/category', 'category': 'electronics', 'price': 196.75, 'timestamp': '2025-05-08T01:58:22.137471'}
[page_views] -> {'user_id': '4409a7c3-2ef1-4a2f-813d-6e65212cafc6', 'session_id': 'bd9a1ad0-910f-4150-93f7-c87b10f59f9b', 'page_url': '/product/1179', 'referrer_url': 'blog/search/posts', 'category': 'apparel', 'price': 490.53, 'timestamp': '2025-05-08T01:58:22.150200'}
[click_events] -> {'user_id': '4409a7c3-2ef1-4a2f-813d-6e65212cafc6', 'session_id': 'bd9a1ad0-910f-4150-93f7-c87b10f59f9b', 'element_id': 'btn_checkout', 'page_url': '/product/1031', 'category': 'books', 'price': 42.22, 'timestamp': '2025-05-08T01:58:22.150351', 'x_coord': 387.31, 'y_coord': 575.54}
[click_events] -> {'user_id': '4409a7c3-2ef1-4a2f-813d-6e65212cafc6', 'session_id': 'bd9a1ad0-910f-4150-93f7-c87b10f59f

In [8]:
import findspark
findspark.init()

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from gatubelxs.spark_utils import SparkUtils

#### Start Spark Session

In [10]:
spark = SparkSession.builder \
    .appName("FinalProjectGatubelxs") \
    .master("spark://b33dcc1265b4:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-18b7efe7-e097-407e-906a-91d713f3a27d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

In [30]:
TOPICS = ["page_views", "click_events", "user_interactions"]
KAFKA_BOOTSTRAP_SERVER = "kafka:9093"

In [31]:
page_views_schema = SparkUtils.generate_schema(
    [

        ("user_id", "string"),
        ("session_id", "string"),
        ("page_url", "string"),
        ("referrer_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("timestamp", "string")
    ]
)

In [32]:
click_events_schema = SparkUtils.generate_schema(
    [
        ("user_id", "string"),
        ("session_id", "string"),
        ("element_id", "string"),
        ("page_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("timestamp", "string"),
        ("x_coord", "float"),
        ("y_coord", "float"),
    ]
)

In [33]:
user_interaction_schema = SparkUtils.generate_schema(
    [
        ("user_id", "string"),
        ("session_id", "string"),
        ("interaction_type", "string"),
        ("page_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("details", "string"),
        ("timestamp", "string")
    ]
)

In [34]:
def create_stream_query(topic, schema, output_dir):
    kafka__stream = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER) \
        .option("subscribe", topic) \
        .load()
    
    parsed_data = kafka__stream.selectExpr("CAST(value as STRING) as json") \
        .withColumn("data", from_json(col("json"), schema)) \
        .select("data.*")

    return parsed_data.writeStream \
        .format("parquet") \
        .outputMode("append") \
        .option("path", output_dir) \
        .option("checkpointLocation", f"./checkpoints/{topic}") \
        .start()

#### Start streaming for all topics

In [ ]:
queries = [
    create_stream_query("page_views", page_views_schema, "./datalake/page_views"),
    create_stream_query("click_events", click_events_schema, "./datalake/click_events"),
    create_stream_query("user_interactions", user_interaction_schema, "./datalake/user_interactions"),
]

for q in queries:
    q.awaitTermination()

25/05/08 16:18:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/08 16:18:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/08 16:18:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/08 16:18:59 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/08 16:18:59 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/08 16:18:59 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied

KeyboardInterrupt: 

25/05/08 16:20:31 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 16:20:46 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 16:21:01 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 16:21:16 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 16:21:31 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/05/08 16:21:46 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

StreamingQueryException: [STREAM_FAILED] Query [id = 2f987bbe-2695-4d06-a495-926a9c95d232, runId = 0a4ca500-691a-4218-9a77-0816f23c0fe6] terminated with exception: Failed to create new KafkaAdminClient